<a href="https://colab.research.google.com/github/seeedata/Dacon/blob/main/%EC%98%A8%EB%9D%BC%EC%9D%B8%20%EC%B1%84%EB%84%90%20%EC%A0%9C%ED%92%88%20%ED%8C%90%EB%A7%A4%EB%9F%89%20%EC%98%88%EC%B8%A1%20AI%20%EC%98%A8%EB%9D%BC%EC%9D%B8%20%ED%95%B4%EC%BB%A4%ED%86%A4/0827%20%EC%B5%9C%EC%A2%85%20%EC%A2%85%ED%95%A9%20%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 불러오기

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/train.csv')

In [ ]:
train_data.head()

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 월별, 계절별 전처리

In [ ]:
cp_data = train_data.copy()

In [ ]:
date_columns = train_data.columns[6:]  # 날짜 컬럼들 선택

# 날짜 수만큼 반복
for col in date_columns:

    if col >= '2022-01-01' and col <= '2023-04-04':

        #2022-01까지 추출
        year_month = col[:7]

        #year_month 열이 데이터셋에 없으면
        if year_month not in train_data.columns:
            train_data[year_month] = 0

        #2022-01에 1월달 값 다 넣기
        train_data[year_month] += train_data[col]
        #이후 반복

In [ ]:
# 계절별 월평균 열 추가, 2023-04는 뺐음
train_data['봄'] = train_data[['2022-03', '2022-04', '2022-05', '2023-03']].mean(axis=1)
train_data['여름'] = train_data[['2022-06', '2022-07', '2022-08']].mean(axis=1)
train_data['가을'] = train_data[['2022-09', '2022-10', '2022-11']].mean(axis=1)
train_data['겨울'] = train_data[['2022-01', '2022-02', '2022-12', '2023-01', '2023-02']].mean(axis=1)
train_data = train_data.drop(['2023-04'], axis = 1)

# 변환된 데이터프레임 출력
print(train_data)

          ID                제품             대분류             중분류  \
0          0  B002-00001-00001  B002-C001-0002  B002-C002-0007   
1          1  B002-00002-00001  B002-C001-0003  B002-C002-0008   
2          2  B002-00002-00002  B002-C001-0003  B002-C002-0008   
3          3  B002-00002-00003  B002-C001-0003  B002-C002-0008   
4          4  B002-00003-00001  B002-C001-0001  B002-C002-0001   
...      ...               ...             ...             ...   
15885  15885  B002-03799-00002  B002-C001-0003  B002-C002-0008   
15886  15886  B002-03799-00003  B002-C001-0003  B002-C002-0008   
15887  15887  B002-03799-00004  B002-C001-0003  B002-C002-0008   
15888  15888  B002-03799-00005  B002-C001-0003  B002-C002-0008   
15889  15889  B002-03799-00010  B002-C001-0002  B002-C002-0004   

                  소분류         브랜드  2022-01-01  2022-01-02  2022-01-03  \
0      B002-C003-0038  B002-00001           0           0           0   
1      B002-C003-0044  B002-00002           0           0    

## 요일별 파생변수(주말만 사용)

In [ ]:
# 열 인덱스들을 날짜 형식으로 변환

date_columns = train_data.columns[6:-19]  # 날짜 컬럼들 선택
date_columns = pd.to_datetime(date_columns)

# 열 이름을 변경하여 날짜 형식으로 열을 추가
cp_data.columns = train_data.columns[:6].tolist() + date_columns.tolist() # 날짜 형식으로 변경하여 추가


# 요일별 합산 열 추가
dayofweek_columns = date_columns.dayofweek  # 요일(0: 월요일, 6: 일요일)
weekday_names = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
for i, day in enumerate(weekday_names):
    train_data[day] = cp_data[date_columns[dayofweek_columns == i]].sum(axis=1)

# 변환된 데이터프레임 출력
print(train_data)

          ID                제품             대분류             중분류  \
0          0  B002-00001-00001  B002-C001-0002  B002-C002-0007   
1          1  B002-00002-00001  B002-C001-0003  B002-C002-0008   
2          2  B002-00002-00002  B002-C001-0003  B002-C002-0008   
3          3  B002-00002-00003  B002-C001-0003  B002-C002-0008   
4          4  B002-00003-00001  B002-C001-0001  B002-C002-0001   
...      ...               ...             ...             ...   
15885  15885  B002-03799-00002  B002-C001-0003  B002-C002-0008   
15886  15886  B002-03799-00003  B002-C001-0003  B002-C002-0008   
15887  15887  B002-03799-00004  B002-C001-0003  B002-C002-0008   
15888  15888  B002-03799-00005  B002-C001-0003  B002-C002-0008   
15889  15889  B002-03799-00010  B002-C001-0002  B002-C002-0004   

                  소분류         브랜드  2022-01-01  2022-01-02  2022-01-03  \
0      B002-C003-0038  B002-00001           0           0           0   
1      B002-C003-0044  B002-00002           0           0    

In [ ]:
#월~금은 없애고 주말만 남김
train_data = train_data.drop(columns = train_data.columns[-7:-2], axis = 1)

## 일자별 제품 금액(판매 가격 / 판매 수량) 평균

In [ ]:
#####상품가 평균값
import pandas as pd
import numpy as np

#---------일자별 제품 금액(판매 가격 / 판매 수량)---------
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/train.csv")  #판매수량
sales = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/sales.csv")  #판매금액

# 날짜 열 선택
date_columns = train.columns[6:]

# 각 날짜별로, 판매 가격을 판매 수량으로 나눈 결과를 저장
# 0으로 나누는 경우를 대비해 결측값을 0으로 채움
goods_price_per_day = sales[date_columns].div(train[date_columns]).fillna(0)

#--------일자별 제품 금액으로 제품의 평균값 도출---------
# 0을 제외한 평균값 계산 (가격이 0인 날은 제외)
goods_price_avg = pd.DataFrame({'평균' : goods_price_per_day.iloc[:, 0:].replace(0, np.nan).mean(axis=1, skipna=True)})

# 모든 날이 0인 경우, 0으로 채우기
train_data['평균'] = goods_price_avg['평균'].fillna(0)

# 출력
train_data.head()

,ID,제품,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,...,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04,평균
0,0,B002-00001-00001,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5856.896552
1,1,B002-00002-00001,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,1,3,2,0,0,2,0,23808.744939
2,2,B002-00002-00002,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12058.598993
3,3,B002-00002-00003,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4010.919540
4,4,B002-00003-00001,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5056.881566


In [ ]:
# 열 순서 변경
col1 = train_data.columns[-1:].to_list()
col2 = train_data.columns[:-1].to_list()
new_col = col1+col2
train_data = train_data[new_col]

In [ ]:
train_data.columns[0]

'평균'

In [ ]:
# 추가로 스케일링할 열 선택
additional_col_to_scale = train_data.columns[0]

# 추가로 스케일링할 열의 min 및 max 계산
additional_min_value = train_data[additional_col_to_scale].min()
additional_max_value = train_data[additional_col_to_scale].max()

# 추가로 스케일링할 열의 범위(max-min) 계산하고, 범위가 0인 경우 1로 대체
additional_range = additional_max_value - additional_min_value
if additional_range == 0:
    additional_range = 1

# 추가로 스케일링할 열의 min-max scaling 수행
train_data[additional_col_to_scale] = (train_data[additional_col_to_scale] - additional_min_value) / additional_range

## 파생변수 조정 코드

In [ ]:
# 열 순서 변경
col1 = train_data.columns[-22:].to_list()
col2 = train_data.columns[:-22].to_list()
new_col = col1+col2
train_data = train_data[new_col]

In [ ]:
train_data.head()

,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,9,3,23,32,0,0,0,1,39,...,0,0,0,0,0,0,0,0,0,0
1,3,0,69,139,73,95,69,38,18,17,...,0,0,0,1,3,2,0,0,2,0
2,0,12,96,211,120,160,156,113,33,51,...,0,0,0,0,0,0,0,0,0,0
3,4,245,140,414,156,212,175,98,39,70,...,0,0,0,0,0,0,0,0,0,0
4,6,25,169,214,2,0,0,0,0,82,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data.columns[:15]

Index(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
       '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03'],
      dtype='object')

In [ ]:
#월 변수들 삭제
train_data = train_data.drop(columns=train_data.columns[:15])

In [ ]:
train_data.columns[13:]

Index(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05',
       '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10',
       ...
       '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
       '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04'],
      dtype='object', length=459)

In [ ]:
train_data.head()

,봄,여름,가을,겨울,토요일,일요일,평균,ID,제품,대분류,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,14.50,0.000000,24.000000,42.6,37,32,5856.896552,0,B002-00001-00001,B002-C001-0002,...,0,0,0,0,0,0,0,0,0,0
1,71.75,67.333333,12.000000,12.2,95,80,23808.744939,1,B002-00002-00001,B002-C001-0003,...,0,0,0,1,3,2,0,0,2,0
2,106.75,143.000000,37.666667,13.6,165,154,12058.598993,2,B002-00002-00002,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
3,177.50,161.666667,47.000000,50.4,248,251,4010.919540,3,B002-00002-00003,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
4,96.25,0.000000,154.666667,75.2,238,170,5056.881566,4,B002-00003-00001,B002-C001-0001,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data.columns[:7]

Index(['봄', '여름', '가을', '겨울', '토요일', '일요일', '평균'], dtype='object')

In [ ]:
# from sklearn.preprocessing import RobustScaler

# # 피생 변수들의 RobustScaler를 생성
# robust_scaler = RobustScaler()

# # 파생 변수들에 대한 데이터 추출
# enocde_cols = train_data.columns[:7]
# encode_data = train_data[enocde_cols]

# # RobustScaler를 사용하여 스케일링
# scaled_holiday_data = robust_scaler.fit_transform(encode_data)

# # 스케일링된 데이터를 다시 원래 데이터프레임에 할당
# train_data[enocde_cols] = scaled_holiday_data

In [ ]:
train_data.head()

,봄,여름,가을,겨울,토요일,일요일,평균,ID,제품,대분류,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,14.50,0.000000,24.000000,42.6,37,32,5856.896552,0,B002-00001-00001,B002-C001-0002,...,0,0,0,0,0,0,0,0,0,0
1,71.75,67.333333,12.000000,12.2,95,80,23808.744939,1,B002-00002-00001,B002-C001-0003,...,0,0,0,1,3,2,0,0,2,0
2,106.75,143.000000,37.666667,13.6,165,154,12058.598993,2,B002-00002-00002,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
3,177.50,161.666667,47.000000,50.4,248,251,4010.919540,3,B002-00002-00003,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
4,96.25,0.000000,154.666667,75.2,238,170,5056.881566,4,B002-00003-00001,B002-C001-0001,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 계절 변수들의 min-max scaling을 수행하는 코드입니다.
season_cols = train_data.columns[:4]
# 칵 column의 min 및 max 계산
min_values = train_data[season_cols].min(axis=1)
max_values = train_data[season_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[season_cols] = (train_data[season_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

In [ ]:
train_data.columns[4:6]

Index(['토요일', '일요일'], dtype='object')

In [ ]:
# 추가로 스케일링할 열 선택
additional_col_to_scale = train_data.columns[4]

# 추가로 스케일링할 열의 min 및 max 계산
additional_min_value = train_data[additional_col_to_scale].min()
additional_max_value = train_data[additional_col_to_scale].max()

# 추가로 스케일링할 열의 범위(max-min) 계산하고, 범위가 0인 경우 1로 대체
additional_range = additional_max_value - additional_min_value
if additional_range == 0:
    additional_range = 1

# 추가로 스케일링할 열의 min-max scaling 수행
train_data[additional_col_to_scale] = (train_data[additional_col_to_scale] - additional_min_value) / additional_range

In [ ]:
# 추가로 스케일링할 열 선택 (예: train_data.columns[6])
additional_col_to_scale = train_data.columns[5]

# 추가로 스케일링할 열의 min 및 max 계산
additional_min_value = train_data[additional_col_to_scale].min()
additional_max_value = train_data[additional_col_to_scale].max()

# 추가로 스케일링할 열의 범위(max-min) 계산하고, 범위가 0인 경우 1로 대체
additional_range = additional_max_value - additional_min_value
if additional_range == 0:
    additional_range = 1

# 추가로 스케일링할 열의 min-max scaling 수행
train_data[additional_col_to_scale] = (train_data[additional_col_to_scale] - additional_min_value) / additional_range

In [ ]:
# 추가로 스케일링할 열 선택 (예: train_data.columns[6])
additional_col_to_scale = train_data.columns[0]

# 추가로 스케일링할 열의 min 및 max 계산
additional_min_value = train_data[additional_col_to_scale].min()
additional_max_value = train_data[additional_col_to_scale].max()

# 추가로 스케일링할 열의 범위(max-min) 계산하고, 범위가 0인 경우 1로 대체
additional_range = additional_max_value - additional_min_value
if additional_range == 0:
    additional_range = 1

# 추가로 스케일링할 열의 min-max scaling 수행
train_data[additional_col_to_scale] = (train_data[additional_col_to_scale] - additional_min_value) / additional_range

In [ ]:
train_data.head()

,봄,여름,가을,겨울,토요일,일요일,평균,ID,제품,대분류,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.340376,0.000000,0.563380,1.000000,0.000077,0.000074,0.007886,0,B002-00001-00001,B002-C001-0002,...,0,0,0,0,0,0,0,0,0,0
1,1.000000,0.926081,0.000000,0.003347,0.000197,0.000184,0.032058,1,B002-00002-00001,B002-C001-0003,...,0,0,0,1,3,2,0,0,2,0
2,0.719861,1.000000,0.185987,0.000000,0.000343,0.000354,0.016237,2,B002-00002-00002,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
3,1.000000,0.878672,0.000000,0.026054,0.000515,0.000577,0.005401,3,B002-00002-00003,B002-C001-0003,...,0,0,0,0,0,0,0,0,0,0
4,0.622306,0.000000,1.000000,0.486207,0.000494,0.000391,0.006809,4,B002-00003-00001,B002-C001-0001,...,0,0,0,0,0,0,0,0,0,0


## 이동평균 데이터 불러오기

In [ ]:
ma_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/이동평균데이터.csv')

In [ ]:
ma_df.head()

,ID,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,2022-01-11,2022-01-12,2022-01-13,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.2,0.8,1.2,1.2,1.2,1.4,0.8
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_data = pd.merge(train_data, ma_df, on = 'ID')

In [ ]:
train_data.head()

,봄,여름,가을,겨울,토요일,일요일,평균,ID,제품,대분류,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.340376,0.000000,0.563380,1.000000,0.000077,0.000074,0.007886,0,B002-00001-00001,B002-C001-0002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.926081,0.000000,0.003347,0.000197,0.000184,0.032058,1,B002-00002-00001,B002-C001-0003,...,0.0,0.0,0.0,0.2,0.8,1.2,1.2,1.2,1.4,0.8
2,0.719861,1.000000,0.185987,0.000000,0.000343,0.000354,0.016237,2,B002-00002-00002,B002-C001-0003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.000000,0.878672,0.000000,0.026054,0.000515,0.000577,0.005401,3,B002-00002-00003,B002-C001-0003,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.622306,0.000000,1.000000,0.486207,0.000494,0.000391,0.006809,4,B002-00003-00001,B002-C001-0001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 모델링전 작업

### 전처리 시작

In [ ]:
train_data.columns[13:]

Index(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05',
       '2022-01-06', '2022-01-07', '2022-01-08', '2022-01-09', '2022-01-10',
       ...
       '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
       '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04'],
      dtype='object', length=459)

In [ ]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[7:]
# 각 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/scale_min_dict.pkl','wb') as f:
  pickle.dump(scale_min_dict,f)

with open('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/scale_max_dict.pkl','wb') as f:
  pickle.dump(scale_max_dict,f)

In [ ]:
brand_keyword_scaled = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/brand_keyword_scaled.csv') # 일별 판매량 Min_Max_Scaled
brand_mean = pd.concat([brand_keyword_scaled['브랜드'], brand_keyword_scaled.iloc[:, 1:].mean(axis = 1)], axis = 1)
brand_mean.columns = ['brand', 'brand_mean']

In [ ]:
### 추가 처리
tmp_br = pd.merge(train_data[['ID', '브랜드']], brand_mean, left_on = '브랜드', right_on = 'brand')
train_data.insert(loc = 5, column = 'brand_mean', value = tmp_br['brand_mean'] )
train_data = train_data.drop(['브랜드'], axis = 1)

### NA값은 전체 평균으로 넣음 (fillna)
train_data['brand_mean'] = train_data['brand_mean'].fillna(train_data['brand_mean'].mean())

In [ ]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류','소분류']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
train_data.tail()

,평균,ID,제품,대분류,중분류,brand_mean,소분류,2022-01-01,2022-01-02,2022-01-03,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
15885,0.002555,15885,B002-03799-00002,2,7,0.306625,41,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15886,0.029507,15886,B002-03799-00003,2,7,0.306625,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.02439,0.0,0.01626,0.03252,0.00813,0.00813,0.024390
15887,0.016247,15887,B002-03799-00004,2,7,0.306625,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000
15888,0.017295,15888,B002-03799-00005,2,7,0.306625,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.142857
15889,0.067054,15889,B002-03799-00010,1,3,0.306625,19,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.00000,0.00000,0.000000


# Modeling - Keras GRU
- 참고: https://www.kaggle.com/code/humamfauzi/multiple-stock-prediction-using-single-nn#Gated-Recurrent-Units

- 데이터 조정 참고: https://dacon.io/competitions/official/236129/codeshare/8668?page=1&dtype=recent

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':60, # 60일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'LEARNING_RATE':5e-4,
    'SEED':9083 # 모델 재현성을 위해 Seed 고정
}

In [ ]:
### Seed 고정 후 진행
import random
import os
import tensorflow as tf

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
train_data.head()

,평균,ID,제품,대분류,중분류,brand_mean,소분류,2022-01-01,2022-01-02,2022-01-03,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.007886,0,B002-00001-00001,1,6,0.177865,37,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,0.032058,1,B002-00002-00001,2,7,0.198931,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,0.016237,2,B002-00002-00002,2,7,0.198931,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,0.005401,3,B002-00002-00003,2,7,0.198931,43,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0.006809,4,B002-00003-00001,0,0,0.013591,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [ ]:
from tqdm.auto import tqdm

### RAM 폭파 대비
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE'],feature_size=3):
    STEP_SIZE = 2 # 이 값을 본인의 환경에 맞게 조정

    num_rows = len(data) #15890
    window_size = train_size + predict_size #81일
    adjusted_size = (len(data.columns) - window_size + 1) // STEP_SIZE #(465-82) // 2

    input_data = np.empty((num_rows * adjusted_size, train_size, len(data.iloc[0, :feature_size]) + 1))
    target_data = np.empty((num_rows * adjusted_size, predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :feature_size])
        sales_data = np.array(data.iloc[i, feature_size:])

        for j in range(0, len(sales_data) - window_size + 1, STEP_SIZE):
            window = sales_data[j: j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * adjusted_size + j // STEP_SIZE] = temp_data
            target_data[i * adjusted_size + j // STEP_SIZE] = window[train_size:]

    return input_data, target_data

In [ ]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'],feature_size=3):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)

    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :feature_size]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :feature_size])
        sales_data = np.array(data.iloc[i, -train_size:])

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def GRU_Model(output_window, inputs,feature_size=3):
    opt = Adam(learning_rate=CFG['LEARNING_RATE'])  # 기존의 학습률 설정 방식으로 변경
    model = Sequential()
    model.add(GRU(128, return_sequences=True, input_shape=(inputs.shape[1], feature_size+1)))  # GRU 레이어의 유닛 수를 늘림
    model.add(GRU(64, return_sequences=False))  # 두 번째 GRU 레이어 추가
    model.add(Dense(64, activation='relu'))  # 추가된 GRU 레이어 뒤에 하나의 Dense 레이어 추가
    model.add(Dropout(0.3))  # 드롭아웃 레이어 추가
    model.add(Dense(output_window, activation='linear'))  # 출력 레이어 유닛 수를 조정
    model.compile(optimizer=opt, loss='mean_squared_error')
    model.summary()
    return model

   model.add(GRU(64, return_sequences = False, input_shape=(inputs.shape[1],feature_size+1)))


   model.add(Dense(32, activation = linear)) 이 부분 줄여보기 고려

## PSFA Score Callback Function

In [ ]:
import numpy as np

indexs_bigcat={}
for bigcat in train_data['대분류'].unique():
    indexs_bigcat[bigcat] = list(train_data.loc[train_data['대분류']==bigcat].index)

def psfa_score(y_true, y_pred):
    PSFA = tf.constant(1.0, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)

    for cat in indexs_bigcat.keys():
        ids = indexs_bigcat[cat]

        for day in range(21):
            total_sell = tf.reduce_sum(tf.gather(y_true, indices=ids, axis=0)[:, day])
            pred_values = tf.gather(y_pred, indices=ids, axis=0)[:, day]
            target_values = tf.gather(y_true, indices=ids, axis=0)[:, day]

            denominator = tf.maximum(target_values, pred_values)
            diffs = tf.where(denominator != 0, tf.abs(target_values - pred_values) / denominator, 0)

            if total_sell != 0:
                    sell_weights = target_values / total_sell
            else:
                sell_weights = tf.ones(shape=(len(ids),), dtype=tf.float32) / tf.cast(len(ids), dtype=tf.float32)

            nan_mask = tf.math.is_nan(diffs)
            if not tf.reduce_any(nan_mask):
                PSFA -= tf.reduce_sum(diffs * sell_weights) / (21 * 5)

    return PSFA

In [ ]:
# 전체 Epoch이 끝날 때 PSFA값을 확인하는 함수.
from tensorflow.keras.callbacks import Callback

class CustomMetricsCallback(Callback):
    def __init__(self, validation_data):
        super(CustomMetricsCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        # 마지막 Epoch에서만 값을 계산하도록 수정 -> 반복적으로 계산하는 경우 RAM이 폭파됨.
        # 여기서 특정 지표 계산 및 출력
        if epoch == self.params['epochs'] - 1:  # 마지막 Epoch에서만 계산
          x_val, y_val = self.validation_data
          predictions = self.model.predict(x_val)

          PSFA_score = psfa_score(y_true = y_val, y_pred = predictions)
          print(f'Validation PSFA at end of epoch {epoch + 1}: {PSFA_score:.4f}')

In [ ]:
import gc

def full_inference(data, output_window = 21, output_file_name = 'set', batch_size = 1024):
    train_input, train_target = make_train_data(data.drop(['ID'], axis = 1))
    test_input = make_predict_data(data.drop(['ID'], axis = 1))

    # Train / Validation Split
    data_len = len(train_input)
    val_s_input = train_input[-int(data_len*0.2):]
    val_s_target = train_target[-int(data_len*0.2):]
    train_s_input = train_input[:-int(data_len*0.2)]
    train_s_target = train_target[:-int(data_len*0.2)]

    # Setting Model
    custom_metrics_callback = CustomMetricsCallback(validation_data = (val_s_input, val_s_target))
    gru_model = GRU_Model(output_window = output_window, inputs = train_input)

    gru_model.fit(train_s_input, train_s_target, validation_data = (val_s_input, val_s_target), epochs = 10, batch_size = 1024,
                  callbacks = [custom_metrics_callback])

    # Prediction
    pred = gru_model.predict(test_input)
    pd.DataFrame(pred).to_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/' + output_file_name + '_pred.csv')
    gru_model.save('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/'+ output_file_name + '_model.h5')

    ### RAM 관리? 를 위해 Input_삭제
    del train_input, train_target, test_input, val_s_input, val_s_target, train_s_input, train_s_target
    gc.collect()

In [ ]:
import gc
gc.collect()

full_inference(data = train_data.drop(['제품','대분류','중분류'], axis = 1), output_window = 21, output_file_name = 'all_set', batch_size = 1024)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 60, 128)           51456     
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 21)                1365      
                                                                 
Total params: 94,229
Trainable params: 94,229
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
2372/2372 [==============================

파라미터 개수가 많으면 모델의 복잡도가 높다고 할 수 있음

##Prediction

### Inverse Scaling

- Scaling 시 얻었던 Min_Max 값들을 Load 후 Inverse Scaling

In [ ]:
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/scale_min_dict.pkl','rb') as file:
    scale_min_dict = pickle.load(file)

with open('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/scale_max_dict.pkl','rb') as file:
  scale_max_dict = pickle.load(file)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/open/sample_submission.csv')
result = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/all_set_pred.csv')
result.columns = submission.columns

In [ ]:
# inverse scaling
from tqdm.auto import tqdm
pred = pd.DataFrame(result)
for idx in range(len(pred)):
    pred.iloc[idx, 1:] = pred.iloc[idx, 1:] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

round_pred = np.round(pred, 0).astype(int)

In [ ]:
round_pred = round_pred.clip(lower = 0)

In [ ]:
round_pred

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,1,2,2,2,2,2,3,3,3,...,3,3,3,3,3,3,4,4,4,4
1,1,6,7,8,8,8,8,8,9,8,...,8,8,8,9,9,9,8,8,8,9
2,2,10,15,19,20,21,23,26,26,25,...,29,30,31,31,31,32,34,34,34,35
3,3,41,63,77,82,87,93,105,105,104,...,117,122,129,126,127,130,140,139,139,145
4,4,9,18,23,26,28,30,35,35,35,...,41,41,43,45,45,49,53,53,54,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,1,2,3,3,3,3,3,3,3,...,4,4,4,4,4,4,4,5,4,5
15886,15886,5,6,6,6,5,6,6,7,6,...,6,6,6,7,7,7,7,6,7,7
15887,15887,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,1
15888,15888,2,2,2,1,1,1,2,2,2,...,1,1,2,2,2,1,1,1,1,2


In [ ]:
# 최종 결과물 저장
round_pred.to_csv('/content/drive/MyDrive/Colab Notebooks/aimers_ai해커톤/submission19.csv', index = False)